In [1]:
##complex data와 똑같이 40ms로 parsing했음 데이터 4배라고 생각하고 뻥튀기시켜봄 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob
import pickle
from sklearn import preprocessing

#filename queue
filename_queue=glob.glob('./rawdata/data[1-6].csv')
#Create return path
return_path='./rawdata/data.csv'
create_file=open(return_path,'w')

In [14]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
index=[]
label_count=np.zeros(17-len(index))
traininglabel_count=np.zeros(17-len(index))
testlabel_count=np.zeros(17-len(index))

df=pd.DataFrame({"labl":[-1],"total":[0.11111]})


#return_file=open(return_path,'a')
#writer=csv.writer(return_file)
#writer.writerow()
n_data=[]

for filename in filename_queue:
    print(1)
    file=open(filename, newline='')
    reader=csv.reader(file)
    header=next(reader)
    
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        date=datetime.strptime(row[0],'%Y/%m/%d')
        msec=row[1:3]
        if(int(row[3])==0):
            labl=0
            label_count[labl]+=1
        if(int(row[3])==1):
            labl=1
            label_count[labl]+=1
        else:
            break
        
        if labl not in index:
            for x in range(4):
                X_acc=list(row[4*i+4+x] for i in range(150))
                Y_acc=list(row[4*i+604+x] for i in range(150))
                Z_acc=list(row[4*i+1204+x] for i in range(150))
                X_gyro=list(row[4*i+1804+x] for i in range(150))
                Y_gyro=list(row[4*i+2404+x] for i in range(150))
                Z_gyro=list(row[4*i+3004+x] for i in range(150))

                #window=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                #    [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                #    [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]],
                #    [X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                #    [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                #    [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])
                
                window=np.array([X_acc[0:25],Y_acc[0:25],Z_acc[0:25],X_gyro[0:25],Y_gyro[0:25],Z_gyro[0:25],
                                  X_acc[25:50],Y_acc[25:50],Z_acc[25:50],X_gyro[25:50],Y_gyro[25:50],Z_gyro[25:50],
                                  X_acc[50:75],Y_acc[50:75],Z_acc[50:75],X_gyro[50:75],Y_gyro[50:75],Z_gyro[50:75],
                                  X_acc[75:100],Y_acc[75:100],Z_acc[75:100],X_gyro[75:100],Y_gyro[75:100],Z_gyro[75:100],
                                  X_acc[100:125],Y_acc[100:125],Z_acc[100:125],X_gyro[100:125],Y_gyro[100:125],Z_gyro[100:125],
                                  X_acc[125:150],Y_acc[125:150],Z_acc[125:150],X_gyro[125:150],Y_gyro[125:150],Z_gyro[125:150]])
                
                
                                
                window_mean=window.mean(axis=-1)
                window_stddev=window.std(axis=-1)
                
                #print('x:',window_stddev[[0,6,12,18,24,30]].mean(axis=0),labl)
                xm=window_stddev[[0,6,12,18,24,30]].mean(axis=0)
                #print('y:',window_stddev[[1,7,13,19,25,31]].mean(axis=0),labl)
                ym=window_stddev[[1,7,13,19,25,31]].mean(axis=0)
                #print('z:',window_stddev[[2,8,14,20,26,32]].mean(axis=0),labl)
                zm=window_stddev[[2,8,14,20,26,32]].mean(axis=0)
                #print('total',np.mean([xm,ym,zm]),'labl:', labl)
                #sort.append([])
                action=np.array([xm,ym])
                print(1)
                #df2= pd.DataFrame({"labl":[labl],"total":[np.mean([xm,ym,zm])]})
                #df=df.append(df2)
                #5 features  what else?2
                
                
                #label_count[labl]+=1

                df.sort_values('total')
                window_median=np.median(window,axis=-1)
                window_percent25=np.percentile(window,25,axis=-1)
                window_percent75=np.percentile(window,75,axis=-1)

                window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
                window_feature=window_feature.reshape(180)

                if (label_count[labl]%5!=1):
                    training_set.append(action)
                    training_label.append(labl)
                    traininglabel_count[labl]+=1
                else:
                    test_set.append(action)
                    test_label.append(labl)
                    testlabel_count[labl]+=1
        

1
123
1
123
1
123
1
123
1
123
1
123


In [3]:
from pyod.models.knn import KNN   # kNN detector
from pyod.utils.data import evaluate_print
from pyod.utils.data import generate_data

X_train=training_set
y_train=training_label
X_test=test_set
y_test=test_label
# train kNN detector
clf_name = 'KNN'
clf = KNN()
clf.fit(X_train)

# get the prediction label and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [4]:

# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)


On Training Data:


NameError: name 'y_train_scores' is not defined

In [5]:
from pyod.utils.example import visualize

visualize(clf_name, X_train, y_train, X_test, y_test, y_train_pred,
    y_test_pred, show_figure=True, save_figure=False)

NameError: name 'y_train_pred' is not defined

In [6]:
# -*- coding: utf-8 -*-
"""Example of using kNN with mahalanobis distance for outlier detection
"""
# Author: Yue Zhao <zhaoy@cmu.edu>
# License: BSD 2 clause

from __future__ import division
from __future__ import print_function

import os
import sys

import numpy as np

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from pyod.models.knn import KNN
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

if __name__ == "__main__":
    contamination = 0.1  # percentage of outliers
    n_train = 200  # number of training points
    n_test = 100  # number of testing points

    # Generate sample data

    # train kNN detector with mahalanobis distance
    clf_name = 'KNN (mahalanobis distance)'
    # calculate covariance for mahalanobis distance
    X_train_cov = np.cov(X_train, rowvar=False)
    clf = KNN(algorithm='auto', metric='mahalanobis',
              metric_params={'V': X_train_cov})
    clf.fit(X_train)

    # get the prediction labels and outlier scores of the training data
    y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    # get the prediction on the test data
    y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
    y_test_scores = clf.decision_function(X_test)  # outlier scores

    # evaluate and print the results
    print("\nOn Training Data:")
    evaluate_print(clf_name, y_train, y_train_scores)
    print("\nOn Test Data:")
    evaluate_print(clf_name, y_test, y_test_scores)

    # visualize the results
    visualize(clf_name, X_train, y_train, X_test, y_test, y_train_pred,
              y_test_pred, show_figure=True, save_figure=False)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: Degrees of freedom <= 0 for slice
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.